In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time

import matplotlib.pyplot as plt
tf.logging.set_verbosity(tf.logging.INFO)


In [14]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

In [15]:
import librosa
s, sr=librosa.load('train_clean_male.wav', sr=None)

S=librosa.stft(s, n_fft=1024, hop_length=512)
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [16]:
import librosa
s, sr=librosa.load('test_x_01.wav', sr=None)

Stest=librosa.stft(s, n_fft=1024, hop_length=512)
# sn, sr=librosa.load('train_dirty_male.wav', sr=None)
# X=librosa.stft(sn, n_fft=1024, hop_length=512)

In [17]:
print(s, sn)

[ 0.00067139 -0.00033569 -0.00045776 ...  0.00027466  0.00067139
  0.00036621] [ 0.00048828  0.00018311  0.00021362 ...  0.0007019  -0.00045776
 -0.00097656]


In [18]:
mag_S=np.abs(S)
mag_X=np.abs(X)
mag_Stest=np.abs(Stest)
print(mag_S.shape, mag_X.shape, mag_Stest.shape)

(513, 2459) (513, 2459) (513, 142)


In [19]:
def weight_variable (shape):
    initial = tf.truncated_normal(shape, stddev = np.sqrt(2.0/sum(shape)) )
    return tf.Variable(initial)

def bias_variable (shape):
#     initial = tf.constant(0.1, shape=shape)
    initial = tf.truncated_normal(shape, stddev = np.sqrt(1.0/sum(shape)) )
    return tf.Variable(initial)

In [20]:
# Create the model
x = tf.placeholder(tf.float32, [None, 513])
# W = tf.Variable(tf.zeros([784, 10]))
# b = tf.Variable(tf.zeros([10]))
# y = tf.matmul(x, W) + b

W_1 = weight_variable([513, 1024])
b_1 = bias_variable([1024])

W_2 = weight_variable([1024, 1024])
b_2 = bias_variable([1024])

W_3 = weight_variable([1024, 1024])
b_3 = bias_variable([1024])

W_4 = weight_variable([1024, 1024])
b_4 = bias_variable([1024])

W_5 = weight_variable([1024, 513])
b_5 = bias_variable([513])

y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)
y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)
y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)
y_4 = tf.nn.relu(tf.matmul(y_3, W_4) + b_4)
y = tf.nn.relu(tf.matmul(y_4, W_5) + b_5) # predicted

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 513]) # original

cross_entropy = tf.losses.mean_squared_error(labels=y_, predictions=y)
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [21]:
sess = tf.InteractiveSession(config=config)
tf.global_variables_initializer().run()

# Train
for _ in range(100):
#     batch_xs, batch_ys = mnist.train.next_batch(1000)
    ts,ce=sess.run([train_step,cross_entropy], feed_dict={x: mag_X.T, y_: mag_S.T})
#     print(ts,ce)

In [22]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
Stest_hat=sess.run(y, feed_dict={x: mag_Stest.T})
S_hat=(Stest/mag_Stest)*Stest_hat.T

In [23]:
Stest_hat.shape
print(S_hat)

[[-0.        +0.        j  0.        +0.        j  0.        +0.        j
  ...  0.        +0.        j  0.        +0.        j
   0.        +0.        j]
 [-0.        +0.        j -0.        +0.        j -0.        +0.        j
  ... -0.        +0.        j  0.        -0.        j
   0.        +0.        j]
 [ 0.        +0.        j -0.        +0.        j  0.        +0.        j
  ...  0.        +0.        j  0.        -0.        j
   0.        -0.        j]
 ...
 [ 0.        +0.        j -0.00417365-0.00144668j -0.01936902+0.00243017j
  ... -0.        +0.        j -0.        +0.        j
  -0.        +0.        j]
 [-0.        +0.        j  0.        +0.        j  0.        +0.        j
  ... -0.        +0.        j  0.        +0.        j
   0.        +0.        j]
 [ 0.        +0.        j -0.        +0.        j -0.02208189+0.        j
  ...  0.        +0.        j -0.        +0.        j
  -0.        +0.        j]]


In [24]:
S_time=librosa.istft(S_hat, hop_length=512)
librosa.output.write_wav('test_s_01_recons.wav', S_time, sr)